In [1]:
!pip install transformers datasets
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.5 MB/s eta 0:00:00


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, DataCollatorWithPadding

/Users/nm/opt/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-06 21:13:28.362832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# All models are a standard torch.nn module so you can use them in any typical training loop
# While you can write your own training loop, Transformers provides a Trainer class for PyTorch
# Trainer class contains the basic training loop and adds additional functionality...
# for features like distributed training, mixed precision, and more.

# To use Trainer for optimised training, start with a PreTrainedModel or a torch.nn.Module
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

# TrainingArguments contains the model hyperparameters you can change 
# like learning rate, batch size, and the number of epochs to train for
current_dir = os.getcwd()
training_args = TrainingArguments(
    output_dir=current_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
)

In [5]:
# Load a preprocessing class like a tokenizer, image processor, feature extractor, or processor
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json: 100%|█████████████████| 28.0/28.0 [00:00<00:00, 51.6kB/s]
vocab.txt: 100%|█████████████████████████████| 232k/232k [00:00<00:00, 8.58MB/s]
tokenizer.json: 100%|████████████████████████| 466k/466k [00:00<00:00, 4.98MB/s]


In [6]:
# Load dataset
dataset = load_dataset("rotten_tomatoes")  # doctest: +IGNORE_RESULT

Generating train split: 100%|█████| 8530/8530 [00:00<00:00, 57801.90 examples/s]
Generating validation split: 100%|█| 1066/1066 [00:00<00:00, 40042.34 examples/s
Generating test split: 100%|██████| 1066/1066 [00:00<00:00, 37434.72 examples/s]


In [7]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

dataset = dataset.map(tokenize_dataset, batched=True)

Map: 100%|████████████████████████| 1066/1066 [00:00<00:00, 40590.90 examples/s]


In [8]:
# A DataCollatorWithPadding helps create a batch of examples from your dataset
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
# Now gather all these classes in Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

In [11]:
# Call train() to start training
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.444200
1000,0.385200
1500,0.271000
2000,0.269100


TrainOutput(global_step=2134, training_loss=0.3358433722555917, metrics={'train_runtime': 324.6636, 'train_samples_per_second': 52.547, 'train_steps_per_second': 6.573, 'total_flos': 195974132394480.0, 'train_loss': 0.3358433722555917, 'epoch': 2.0})